In [28]:
import tensorflow as tf
import time

In [29]:
class FileDataset(tf.data.Dataset):
    def read_file_in_batches(num_samples):
        # Opening the file
        time.sleep(0.03)

        for sample_idx in range(num_samples):
            # Reading data (line, record) from the file
            time.sleep(0.015)

            yield (sample_idx,)

    def __new__(cls, num_samples=3):
        return tf.data.Dataset.from_generator(
            cls.read_file_in_batches,
            output_signature = tf.TensorSpec(shape = (1,), dtype = tf.int64),
            args=(num_samples,)
        )

In [30]:
def benchmark(dataset, num_epochs=2):
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performing a training step
            time.sleep(0.01)

In [36]:
%%timeit
benchmark(FileDataset())

289 ms ± 5.18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%%timeit
benchmark(FileDataset().prefetch(1)) # Prefetch is Parallel Working of CPU and GPU

288 ms ± 2.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

287 ms ± 851 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Cache saves time by only reading and opening the file once in first epoch and ignore for rest

In [49]:
dataset = tf.data.Dataset.range(5)
dataset = dataset.map(lambda x: x**2)

In [50]:
dataset = dataset.cache("mycache.txt")
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [52]:
def mapped_function(s):
    tf.py_function(lambda: time.sleep(0.03), [], ())
    return s

In [53]:
%%timeit -r1 -n1
benchmark(FileDataset().map(mapped_function), 5)

1.29 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [54]:
%%timeit -r1 -n1
benchmark(FileDataset().map(mapped_function).cache(), 5)

472 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
